In [1]:
import sys
import os
import numpy as np
import renders as rs
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from IPython.display import display # Allows the use of display() for DataFrames
from sklearn.externals import joblib
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.cross_validation import StratifiedKFold
from sklearn.svm import SVC
import matplotlib.cm as cm
# Show matplotlib plots inline (nicely formatted in the notebook)
%matplotlib inline

# Might need to change the path of the included libraries.
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/match_stats.py')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/form_model.py')
sys.path.append('/anaconda/envs/stats/lib/python3.5/site-packages')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/model_libs.py')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/form_data.py')
sys.path.append('/Users/senzari/Machine_Learning/stats')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats')
# print(sys.path)
os.chdir('/Users/senzari/Machine_Learning/stats/src')
#print(os.getcwd())

from stats import form_data, match_stats, model_libs, form_model, predict_matches, match_stats_alternate

# Variables
round_number = 27 # for MLS only
target_col = 'points'
ignore_cols = ['match_id', 'team_id', 'team_name', 'opp_id', 'opp_name', 'scheduled', 'games_played', 'round']
sub_cols = ['current_formation', 'avg_goals_against', 'goal_diff', 'win_percentage', 'sos',
           'opp_win_percentage', 'opp_sos', 'current_team_yellow_cards', 'current_team_corner_kicks', 'current_team_first_half_goals', 'current_team_sec_half_goals', 
           'opp_team_yellow_cards', 'opp_team_corner_kicks', 'opp_team_first_half_goals', 'opp_team_sec_half_goals']

all_models = ['log', 'svc', 'gmm', 'knn', 'gnb', 'randomForest']

""" this variable 'testing' should be false if using CSV's and not pulling from the database. """
testing = False

INITIALIZED...


In [2]:
data_csv = 'raw_data_alternate.csv'

if testing:
    raw_data = form_data.run_data()
    raw_data.to_csv(data_csv)
    print("Raw Data Saved to CSV")
else:
    #Reading in a CSV adds the first index column
    raw_data = pd.read_csv(data_csv)
    raw_data = raw_data.drop(raw_data.columns[[0]], axis=1)

pd.set_option("display.max_columns", 85)
print('Data Loaded...')
print("Dataset size :: {}".format(raw_data.shape))
display(raw_data.head())

Data Loaded...
Dataset size :: (634, 22)


,match_id,team_id,team_name,opp_id,opp_name,scheduled,round,games_played,is_home,current_formation,diff_goal_for,diff_goal_allowed,diff_attacks,diff_dangerous_attacks,diff_goal_attempts,diff_ball_safe,goals_for,goals_allowed,rpi,opp_rpi,goals,points
0,249,21,FC Dallas,33,DC United,2016-03-26 21:30:00,4,3,0,4-2-3-1,12,0,10251,10624,741,-14553,4,5,0.630952,0.477444,3,3
1,255,21,FC Dallas,39,Columbus Crew,2016-04-03 01:00:00,5,3,1,4-2-3-1,21,9,29375,-9075,-497,19152,5,5,0.465919,0.407233,1,1
2,265,21,FC Dallas,32,San Jose Earthquakes,2016-04-10 01:00:00,6,3,1,4-4-2,20,-24,19440,5423,1664,-13068,6,1,0.547723,0.456140,2,1
3,272,21,FC Dallas,30,Portland Timbers,2016-04-14 02:30:00,7,3,0,4-4-2,20,-40,6100,-5871,1105,-1629,6,3,0.497669,0.573715,3,3
4,282,21,FC Dallas,27,Sporting Kansas City,2016-04-17 23:00:00,7,3,1,4-4-2,20,5,-4389,-2079,1197,-14716,6,3,0.497669,0.523888,2,3


## FORMATTING

In [26]:
# Helper Function - Removes Columns to Ignore and Splits the Target Column
def split_target(data):
    td = model_libs._clone_and_drop(data, ignore_cols)
    (y, X) = model_libs._extract_target(td, target_col)
    return X, y

""" Need to do some formatting of the Data before we run the models"""
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1000)

rankings_data = raw_data

rankings_data["offensive_ranking"] = pd.Series(None, index=rankings_data.index)
rankings_data["opp_defensive_ranking"] = pd.Series(None, index=rankings_data.index)
rankings_data["rpi_ranking"] = pd.Series(None, index=rankings_data.index)
rankings_data["opp_rpi_ranking"] = pd.Series(None, index=rankings_data.index)

leagues = model_libs.get_leagues_country_codes()
teams = form_data.get_teams()
league_rounds = model_libs.get_leagues_rounds()
test = True
if test:
    """ Going through each League"""
    for key, value in leagues.iteritems():
        print(key)
        country_code = leagues[key]
        round_num = league_rounds[key]
        #round_num = 6
        teams_in_league = teams[teams["country_code"] == country_code]
        """ Looping through the Rounds """
        for i in range(4, round_num):
            
            print("ROUND :: {} ".format(i))
            
            rpi_rankings = form_data.get_rankings(teams_in_league, i, "rpi", False)
            r_rankings = model_libs.quartile_list(rpi_rankings, True)
            rpi_rankings["rpi_rankings_quartiled"] = r_rankings
            #print(rpi_rankings)
            print("Finished with RPI Rankings")
        
            offensive_rankings = form_data.get_rankings(teams_in_league, i, "offensive", False)
            rankings = model_libs.quartile_list(offensive_rankings, True)
            offensive_rankings["offensive_rankings_quartiled"] = rankings
            print("Finished with Offensive Rankings")
            #print(offensive_rankings)

            defensive_rankings = form_data.get_rankings(teams_in_league, i, "defensive", False)
            rankings = model_libs.quartile_list(defensive_rankings, False)
            defensive_rankings["defensive_rankings_quartiled"] = rankings
            print("Finished with Defensive Rankings")
            #print(defensive_rankings)

            """ Loop through each Team in the League for that round and assign an Offensive Rank """
            for key, team in teams_in_league.iterrows():
                
                ''' Assigning RPI Rankings to the Current Team and the Opponent Team '''
                rpi_rank = rpi_rankings.loc[rpi_rankings[0] == team['id'], "rpi_rankings_quartiled"]
                
                r_idx = rankings_data.loc[(rankings_data["team_id"] == team["id"]) 
                        & (rankings_data["round"] == (i)), "rpi_ranking"].index
               
                opp_r_idx = rankings_data.loc[(rankings_data["opp_id"] == team["id"]) 
                        & (rankings_data["round"] == (i)), "rpi_ranking"].index

                rankings_data.set_value(r_idx, "rpi_ranking", rpi_rank.values[0])
                rankings_data.set_value(opp_r_idx, "opp_rpi_ranking", rpi_rank.values[0])

                ''' If the team is the team_id then put in their offensive ranking for that game '''
                offensive_rank = offensive_rankings.loc[offensive_rankings[0] == team['id'], "offensive_rankings_quartiled"]
                idx = rankings_data.loc[(rankings_data["team_id"] == team["id"]) 
                        & (rankings_data["round"] == (i)), "offensive_ranking"].index

                rankings_data.set_value(idx, "offensive_ranking", offensive_rank.values[0])
                
                ''' If the team is the opp then put in their defensive ranking for that game '''
                defensive_rank = defensive_rankings.loc[defensive_rankings[0] == team['id'], "defensive_rankings_quartiled"]
                opp_idx = rankings_data.loc[(rankings_data["opp_id"] == team["id"]) 
                        & (rankings_data["round"] == (i))].index

                rankings_data.set_value(opp_idx, "opp_defensive_ranking", defensive_rank.values[0])
                
                #display(rankings_data.head(10))
                
            rankings_data.to_csv('rankings_data_alternate.csv')
                
else:
    
    rankings_data = pd.read_csv('rankings_data_alternate.csv')
    rankings_data = rankings_data.drop(rankings_data.columns[[0]], axis=1)
    
print('Data Loaded...')
                          
""" Formatting data to convert goals scored to the correct category"""
# Not using points as a target for this version, using goals
rankings_data = rankings_data.drop('goals', 1)

#rankings_data['converted_goals'] = rankings_data.apply(lambda row: model_libs.set_group(row['goals']), axis=1)

primera_division
ROUND :: 4 
Rankings :: rpi
Finished with RPI Rankings
Rankings :: offensive
Finished with Offensive Rankings
Rankings :: defensive
Finished with Defensive Rankings
mls
ROUND :: 4 
Rankings :: rpi
Finished with RPI Rankings
Rankings :: offensive
Finished with Offensive Rankings
Rankings :: defensive
Finished with Defensive Rankings
ROUND :: 5 
Rankings :: rpi
Finished with RPI Rankings
Rankings :: offensive
Finished with Offensive Rankings
Rankings :: defensive
Finished with Defensive Rankings
ROUND :: 6 
Rankings :: rpi
Finished with RPI Rankings
Rankings :: offensive
Finished with Offensive Rankings
Rankings :: defensive
Finished with Defensive Rankings
ROUND :: 7 
Rankings :: rpi
Finished with RPI Rankings
Rankings :: offensive
Finished with Offensive Rankings
Rankings :: defensive
Finished with Defensive Rankings
ROUND :: 8 
Rankings :: rpi
Finished with RPI Rankings
Rankings :: offensive
Finished with Offensive Rankings
Rankings :: defensive
Finished with Defensiv

In [42]:
rankings_data = pd.read_csv('rankings_data_alternate.csv')
rankings_data = rankings_data.drop(rankings_data.columns[[0]], axis=1)

### RUNNING CLASSIFICATION MODEL

In [43]:
def run_features(data, drop_data, target, models):
    
    new_data = data.drop(drop_data, axis=1)
    
    #display(new_data.head())
    
    (y, X) = model_libs._extract_target(new_data, target)
    
    models = form_model.train_models(round_number, X, y, models)
    
    return models

#rankings_data = rankings_data.drop(['rpi', 'opp_rpi'], 1)
columns_to_drop = ['current_formation', 'goals', 'rpi', 'opp_rpi', 'offensive_ranking', 'opp_defensive_ranking']
rankings_data = rankings_data.drop(ignore_cols + columns_to_drop, 1)

#### Running ALL Features 
models_test_1 = run_features(rankings_data, [], 'points', ["knn"])

(rankings_y, rankings_X) = model_libs._extract_target(rankings_data, 'points')

def check_accuracy(model, data_X):
    actual_y = pd.DataFrame(rankings_y.values, columns=['actual'])
    predictions = pd.DataFrame(model.predict(data_X), columns=['predictions'])
    preds = pd.concat([predictions, actual_y], axis=1)
    preds['diff'] = preds.apply(lambda r: model_libs.predictions_diff(r['predictions'], r['actual']), axis=1)
    accuracy = np.divide(preds['diff'].sum(), float(len(preds['diff'])))
    print(accuracy)

for m in models_test_1:
    check_accuracy(m, rankings_X)

-----------------------------------
Training K Neighbors Classifier Model
KNN Score on Training Set :: 0.631163708087
KNN Score on Test Set:: 0.267716535433
Finished K-Means Modeling
0.558359621451


In [ ]:
print(rankings_X.columns)

In [ ]:
def train_tuned_models(round_num, X, y):
    for i in all_models:
        models = form_model.build_tuned_model(X, y, i)
        
    return models
        
tuned_models = train_tuned_models(round_number, classifier_X, classifier_y)

In [30]:
log_model = form_model.build_tuned_model(rankings_X, rankings_y, 'log')

for m in log_model:
    print(m)
    check_accuracy(m, rankings_X)

Training LOG REG Model
Score on Training Set :: 0.449704142012
Score on Test Set :: 0.417322834646
Finished LOG REG Modeling
LogisticRegression(C=100000.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)
0.443217665615


In [31]:
random_forest_model = form_model.build_tuned_model(rankings_X, rankings_y, 'randomForest')

for m in random_forest_model:
    print(m)
    check_accuracy(m, rankings_X)

-----------------------------------
Training Random Forest Model
[ 0.3592233   0.42156863  0.3960396   0.40594059  0.43      ]
Accuracy: 0.40 (+/- 0.05)
Finished Random Forest Modeling
RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=5, max_features=2,
            max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=11, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
0.582018927445


In [32]:
knn_model = form_model.build_tuned_model(rankings_X, rankings_y, 'knn')

for m in knn_model:
    check_accuracy(m, rankings_X)

-----------------------------------
Training K-Means Model
KNN Score :: 0.641584158416 for Training
KNN Score :: 0.286821705426 for Testing
KNN Score :: 0.615384615385 for Training
KNN Score :: 0.259842519685 for Testing
KNN Score :: 0.641732283465 for Training
KNN Score :: 0.301587301587 for Testing
KNN Score :: 0.616141732283 for Training
KNN Score :: 0.31746031746 for Testing
KNN Score :: 0.604330708661 for Training
KNN Score :: 0.277777777778 for Testing
Finished K-Means Modeling
0.539432176656


In [33]:
gnb_model = form_model.build_tuned_model(rankings_X, rankings_y, 'gnb')

for m in gnb_model:
    print(m)
    check_accuracy(m, rankings_X)

-----------------------------------
Training Gaussian NB Model
[ 0.34951456  0.37254902  0.38613861  0.37623762  0.37      ]
Accuracy: 0.37 (+/- 0.02)
Finished Gaussian NB Modeling
GaussianNB()
0.451104100946


In [34]:
svc_model = form_model.build_tuned_model(rankings_X, rankings_y, 'svc')

display(rankings_X.columns)
for m in svc_model:
    print(m)
    check_accuracy(m, rankings_X)

Training and Tuning SVC Model
[ 0.34251969  0.33596838]
Accuracy: 0.34 (+/- 0.01)
Finished SVC Modeling


Index([u'is_home', u'diff_goal_for', u'diff_goal_allowed', u'diff_attacks',
       u'diff_dangerous_attacks', u'diff_goal_attempts', u'diff_ball_safe',
       u'goals_for', u'goals_allowed', u'rpi_ranking', u'opp_rpi_ranking'],
      dtype='object')

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'kernel': ['rbf'], 'C': [1, 10, 100]}],
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)
0.859621451104


In [35]:
gmm_model = form_model.build_tuned_model(rankings_X, rankings_y, 'gmm')

for m in gmm_model:
    print(m)
    check_accuracy(m, rankings_X)

-----------------------------------
Training and Tuning GMM Model
W/ Covariance Type :: spherical
# of Components :: 2
Silhouette Score :: 0.407006824962 for Training
Silhouette Score :: 0.396810765839 for Testing
# of Components :: 3
Silhouette Score :: 0.0544413529268 for Training
Silhouette Score :: 0.128576522465 for Testing
# of Components :: 4
Silhouette Score :: 0.125228401255 for Training
Silhouette Score :: 0.210759103172 for Testing
W/ Covariance Type :: tied
# of Components :: 2
Silhouette Score :: 0.362692049782 for Training
Silhouette Score :: 0.370609352856 for Testing
# of Components :: 3
Silhouette Score :: 0.287343615079 for Training
Silhouette Score :: 0.308163981265 for Testing
# of Components :: 4
Silhouette Score :: -0.0431281845727 for Training
Silhouette Score :: -0.0428170625913 for Testing
W/ Covariance Type :: diag
# of Components :: 2
Silhouette Score :: 0.301166874724 for Training
Silhouette Score :: 0.274330394499 for Testing
# of Components :: 3
Silhouette

Cross Validating the SVC model with the PCA data to help prevent overfitting

In [36]:
prediction_models = form_model.load_models(['knn', 'svc', 'randomForest', 'gnb', 'gmm', 'log'])

Success :: Loaded - knn
Success :: Loaded - svc
Success :: Loaded - randomForest
Success :: Loaded - gnb
Success :: Loaded - gmm
Success :: Loaded - log


In [39]:
print('Upcoming matches')
#upcoming_matches, match_details = predict_matches.get_upcoming_matches()
#upcoming_matches.to_csv('upcoming_matches.csv')
upcoming_matches = pd.read_csv('upcoming_matches.csv')
upcoming_matches = upcoming_matches.drop(upcoming_matches.columns[[0]], axis=1)
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)
upcoming_data = predict_matches.predictions(upcoming_matches)
#display(upcoming_data.head())
print('Data Loaded and Predicted...')

upcoming_data["offensive_ranking"] = pd.Series(None, index=rankings_data.index)
upcoming_data["opp_defensive_ranking"] = pd.Series(None, index=rankings_data.index)
upcoming_data["rpi_ranking"] = pd.Series(None, index=rankings_data.index)
upcoming_data["opp_rpi_ranking"] = pd.Series(None, index=rankings_data.index)

leagues = model_libs.get_leagues_country_codes()
teams = form_data.get_teams()
league_rounds = model_libs.get_leagues_rounds()
test = False
if test:
    """ Going through each League"""
    for key, value in leagues.iteritems():
        print(key)
        country_code = leagues[key]
        round_num = league_rounds[key]
        teams_in_league = teams[teams["country_code"] == country_code]
        
        print("ROUND :: {} ".format(round_num))
        
        rpi_rankings = form_data.get_rankings(teams_in_league, round_num, "rpi", True)
        r_rankings = model_libs.quartile_list(rpi_rankings, True)
        rpi_rankings["rpi_rankings_quartiled"] = r_rankings
        #print(rpi_rankings)
        print("Finished with RPI Rankings")
        
        offensive_rankings = form_data.get_rankings(teams_in_league, round_num, "offensive", True)
        o_rankings = model_libs.quartile_list(offensive_rankings, True)
        offensive_rankings["offensive_rankings_quartiled"] = o_rankings
        #print(offensive_rankings)
        print("Finished with Offensive Rankings")

        defensive_rankings = form_data.get_rankings(teams_in_league, round_num, "defensive", True )
        d_rankings = model_libs.quartile_list(defensive_rankings, False)
        defensive_rankings["defensive_rankings_quartiled"] = d_rankings
        #print(defensive_rankings)
        print("Finished with Defensive Rankings")

        """ Loop through each Team in the League for that round and assign an Offensive/Defensive Rank """
        for key, team in teams_in_league.iterrows():
            
            ''' Assigning RPI Rankings to the Current Team and the Opponent Team '''
            rpi_rank = rpi_rankings.loc[rpi_rankings[0] == team['id'], "rpi_rankings_quartiled"]
            r_idx = upcoming_data.loc[(upcoming_data["team_id"] == team["id"]) 
                    & (upcoming_data["round"] == (round_num)), "rpi_ranking"].index
            opp_r_idx = upcoming_data.loc[(upcoming_data["opp_id"] == team["id"]) 
                    & (upcoming_data["round"] == (round_num)), "rpi_ranking"].index
            
            upcoming_data.set_value(r_idx, "rpi_ranking", rpi_rank.values[0])
            upcoming_data.set_value(opp_r_idx, "opp_rpi_ranking", rpi_rank.values[0])
            
            ''' If the team is the team_id then put in their offensive ranking for that game '''
            offensive_rank = offensive_rankings.loc[offensive_rankings[0] == team['id'], "offensive_rankings_quartiled"]
            idx = upcoming_data.loc[(upcoming_data["team_id"] == team["id"]) 
                    & (upcoming_data["round"] == (round_num)), "offensive_ranking"].index
            upcoming_data.set_value(idx, "offensive_ranking", offensive_rank.values[0])
            
            ''' If the team is the opp then put in their defensive ranking for that game '''
            defensive_rank = defensive_rankings.loc[defensive_rankings[0] == team['id'], "defensive_rankings_quartiled"]
            opp_idx = upcoming_data.loc[(upcoming_data["opp_id"] == team["id"]) 
                    & (upcoming_data["round"] == (round_num))].index

            upcoming_data.set_value(opp_idx, "opp_defensive_ranking", defensive_rank.values[0])
                
        upcoming_data.to_csv('upcoming_formatted_matches_alternate.csv')
                
else:
    
    upcoming_data = pd.read_csv('upcoming_formatted_matches_alternate.csv')
    upcoming_data = upcoming_data.drop(upcoming_data.columns[[0]], axis=1)



print('Added Rankings to Upcoming Matches')

Upcoming matches
Data Loaded and Predicted...
Added Rankings to Upcoming Matches


In [49]:
""" Need to remove the same columns from the data the same way we did on the raw data """

upcoming_formatted_data = upcoming_data.drop(ignore_cols + ['current_formation', 'goals', 'rpi', 'offensive_ranking', 'opp_defensive_ranking'], 1)
upcoming_formatted_data = upcoming_formatted_data.drop('points', 1)
display(upcoming_formatted_data.head())

,is_home,diff_goal_for,diff_goal_allowed,diff_attacks,diff_dangerous_attacks,diff_goal_attempts,diff_ball_safe,goals_for,goals_allowed,rpi_ranking,opp_rpi_ranking
0,0,16,-9,-18960,-8027,365,-25725,5,4,1.00000,0.00000
1,0,-8,-5,5780,3780,135,9231,1,2,0.33333,0.66666
2,1,-16,9,18960,8027,-365,25725,3,5,0.00000,1.00000
3,1,0,33,25852,6187,-57,28717,6,7,0.66666,0.33333
4,0,-21,11,-1272,-24480,0,-579,2,6,0.33333,1.00000


In [50]:
""" Models we'll use to predict on upcoming matches """
# pca_svc_model, knn_model, random_forest_model

# This is all the X values
upcoming_formatted_data

svc_preds = prediction_models[1].predict(upcoming_formatted_data)
#print(svc_decsions)

rf_preds = prediction_models[2].predict(upcoming_formatted_data)
print(rf_preds)
knn_preds = prediction_models[0].predict(upcoming_formatted_data)
print(knn_preds)

gmm_preds = prediction_models[3].predict(upcoming_formatted_data)
print(gmm_preds)

gnb_preds = prediction_models[4].predict(upcoming_formatted_data)
print(gnb_preds)

log_preds = prediction_models[5].predict(upcoming_formatted_data)
log_prob = prediction_models[5].predict_proba(upcoming_formatted_data)
probs = pd.DataFrame(log_prob)
print(probs)
print(log_preds)


[ 0.  3.  1.  3.  1.  3.  0.  3.  0.  0.  3.  3.  3.  3.  0.  3.  0.  3.
  1.  0.  1.  3.  3.  1.  1.  0.  0.  1.  0.  0.  1.  0.  1.  3.  3.  3.
  3.  1.  1.  3.  0.  3.  0.  0.  0.  1.  3.  3.  3.  3.  0.  3.  3.  0.
  0.  3.  0.  3.  3.  0.  0.  0.  1.  3.  3.  3.  0.  3.  3.  1.  0.  3.
  3.  3.  0.  0.  0.  1.  1.  1.  0.  1.  0.  0.  3.  3.  1.  0.  3.  0.
  0.  3.  3.  1.  3.  1.  0.  1.]
[ 1.  0.  0.  1.  0.  0.  0.  3.  1.  1.  0.  3.  3.  3.  0.  0.  1.  0.
  3.  1.  1.  3.  0.  1.  0.  0.  0.  3.  1.  0.  0.  0.  0.  1.  1.  0.
  3.  1.  0.  0.  0.  1.  0.  1.  1.  1.  0.  0.  0.  0.  3.  0.  0.  0.
  1.  0.  0.  3.  1.  3.  0.  0.  0.  1.  0.  0.  0.  0.  1.  0.  1.  0.
  1.  1.  0.  0.  1.  0.  3.  0.  0.  1.  0.  0.  1.  0.  0.  3.  1.  3.
  0.  1.  0.  0.  0.  0.  0.  0.]
[ 1.  1.  1.  0.  0.  1.  0.  1.  0.  3.  3.  3.  3.  3.  0.  3.  1.  0.
  0.  1.  3.  3.  1.  3.  0.  3.  0.  1.  0.  0.  0.  0.  0.  3.  0.  3.
  3.  3.  3.  0.  3.  0.  0.  1.  0.  3.  3.  3.  3.  0.

In [53]:
columns = ['team_name', 'opp_name', 'scheduled']
# Remove all columns except the ones above
#upcoming_matches = upcoming_data[columns]
upcoming_matches = upcoming_data

display(probs.head(1))

#random_preds = pd.Series(np.random.randint(3, size=len(upcoming_matches.index)), upcoming_matches.index)
#random_preds[random_preds == 2] = 3

# Add predictions to the end of that DF
results = pd.DataFrame({'KNN': knn_preds, 'RandomForest': rf_preds, 'GNB': gnb_preds, 'GMM': gmm_preds, 'log': log_preds, 'SVC': svc_preds})

upcoming_matches = pd.concat([upcoming_matches, results, probs], axis = 1)
display(upcoming_matches.head(1))
reordered_matches = pd.DataFrame([])

for rows in upcoming_matches.iterrows():
    for i in upcoming_matches['team_name']:
        if rows[1]['opp_name'] == i:
            reordered_matches = reordered_matches.append(rows[1])
            reordered_matches = reordered_matches.append(upcoming_matches[upcoming_matches['team_name'].isin([i])])

reordered_matches = reordered_matches.drop_duplicates() 
columns = ['scheduled', 'team_name', 'opp_name', 'is_home', 'diff_goal_for', 'diff_goal_allowed', 'diff_attacks', 
           'diff_dangerous_attacks', 'diff_goal_attempts', 'diff_ball_safe', 
           'rpi_ranking', 'opp_rpi_ranking','KNN', 'RandomForest', 'GNB', 'GMM', 'SVC', 'log', 0, 1, 2]
reordered_matches = reordered_matches[columns]
reordered_matches.to_csv('predictions_on_upcoming_alternate.csv')
print('Prediction CSV saved')

,0,1,2
0,0.357374,0.333181,0.309445


,match_id,team_id,team_name,opp_id,opp_name,scheduled,round,games_played,is_home,current_formation,diff_goal_for,diff_goal_allowed,diff_attacks,diff_dangerous_attacks,diff_goal_attempts,diff_ball_safe,rpi,goals_for,goals_allowed,goals,points,offensive_ranking,opp_defensive_ranking,rpi_ranking,opp_rpi_ranking,GMM,GNB,KNN,RandomForest,SVC,log,0,1,2
0,900,21,FC Dallas,23,Real Salt Lake,2016-09-25 01:30:00,28,3,0,4-2-3-1,16,-9,-18960,-8027,365,-25725,0.506589,5,4,0,0,1,1,1,0,1,1,1,0,3,0,0.357374,0.333181,0.309445


Prediction CSV saved


In [ ]:
actual_data = pd.read_csv('predictions_on_upcoming_alternate_no_offensive_actual.csv')
actual_data = actual_data.drop(actual_data.columns[[0]], axis=1)
display(actual_data.head(1))
actual_data['diff1'] = actual_data.apply(lambda r: model_libs.predictions_diff(r['actual'], r['KNN']), axis=1)
accuracy = np.divide(actual_data['diff1'].sum(), float(len(actual_data['diff1'])))
print(accuracy)

In [ ]:
abbreviated_data = actual_data.drop(["KNN", "RandomForest", "GNB", "GMM", "random"], axis=1)
print(abbreviated_data.shape)
display(abbreviated_data)